In [7]:
from pydrake.all import (
    GraphOfConvexSets,
    GraphOfConvexSetsOptions,
    Cost,
    Constraint,
    Binding,
    MathematicalProgramResult,
    Point
)
import numpy as np
from large_gcs.graph.cost_factory import create_l2norm_edge_cost


gcs = GraphOfConvexSets()
source_set = Point([0, 0])
mid_set = Point([1, 1])
target_set = Point([0, 9])
gcs_edges = []
v_t = gcs.AddVertex(target_set, "t")
v_s = gcs.AddVertex(source_set, "s")
v_m = gcs.AddVertex(mid_set, "m")
edges = [(v_s, v_m), (v_m, v_t)]

cost = create_l2norm_edge_cost(2)
for edge in edges:
    e = gcs.AddEdge(edge[0], edge[1])
    gcs_edges.append(e)
    x = np.array([e.xu(), e.xv()]).flatten()
    binding = Binding[Cost](cost, x)
    e.AddCost(binding)

options = GraphOfConvexSetsOptions()
options.convex_relaxation = False

result = gcs.SolveShortestPath(
            v_s,
            v_t,
            options,
        )
assert result.is_success()
print(result.get_optimal_cost())


9.476471310671645


In [1]:
from large_gcs.example_graphs.spp_gcs import create_spp_2d_graph
from large_gcs.graph.cost_factory import create_l2norm_edge_cost
from large_gcs.graph.graph import Edge, Graph

In [6]:
G = create_spp_2d_graph(create_l2norm_edge_cost)
G2 = Graph(G._default_costs_constraints)
G2.add_vertex(G.vertices[G.target_name], G.target_name)
G2.add_vertex(G.vertices[G.source_name], G.source_name)
m_name = "p0"
G2.add_vertex(G.vertices[m_name], m_name)
G2.set_target(G.target_name)
G2.set_source(G.source_name)

G2.add_edge(G.edges[(G.source_name, m_name)])

direct_to_target = Edge(m_name, G.target_name)
G2.add_edge(direct_to_target)

G2.solve_shortest_path()


ShortestPathSolution(cost=8.999999945617564, time=0.001719991210848093, path=[('s', array([0., 0.])), ('p0', array([2.06711309e+00, 2.44716369e-04])), ('t', array([9., 0.]))], flows=[1.0, 1.0], result=<pydrake.solvers.MathematicalProgramResult object at 0x7f900d5bd8f0>)